Install lxml for web scraping

In [9]:
!pip3 install lxml --user

     |████████████████████████████████| 5.4 MB 5.0 MB/s eta 0:00:01     |████████████████████████████▏   | 4.8 MB 5.0 MB/s eta 0:00:01


In [34]:
import pandas as pd

The following cell reads the tabular data from the specified url and converts it into a pandas dataframe

In [35]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)
df = pd.DataFrame(df_list[0])
print("Shape before ", df.shape)
df.head()

Shape before  (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Replace 'Not assigned' neighbourhood with corresponding Borough value

In [36]:
for index, row in df.iterrows():
    if df.iloc[index]['Neighbourhood'] == 'Not assigned':
        df.iloc[index]['Neighbourhood'] = df.iloc[index]['Borough']

print("Shape ", df.shape)
df.head()

Shape  (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove rows with Borough = 'Not assigned'

In [37]:
filtered_df = df[df.Borough != 'Not assigned']
filtered_df = filtered_df.reset_index()
del filtered_df['index']
print("Shape after ", filtered_df.shape)
filtered_df.head(10)

Shape after  (103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"
